In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd drive/MyDrive/HCI

In [183]:
ls

Dataset.txt  Terrorism_Related_Words.xlsx


In [108]:
# Reading the Dataset
file_path = 'Dataset.txt'

with open(file_path, 'r') as file:
    file_content = file.read()

In [109]:
import pandas as pd
import re

# Splitting the content into individual reports
reports = file_content.split("REPORT\n")[1:]  # [1:] to skip the first empty split

# Function to extract data from a report based on the given keys
def extract_report_data(report):
    keys = ['ID', 'REPORTDATE', 'REFERENCEID', 'REPORTSOURCE', 'REPORTDESCRIPTION', 'PERSONS', 'DATES', 'PLACES', 'ORGANIZATIONS']
    report_data = {}

    for key in keys:
        # Using regular expression to find the corresponding value for each key
        pattern = f"{key}: (.*?)\n"
        match = re.search(pattern, report, re.S)
        report_data[key] = match.group(1).strip() if match else None

    return report_data

# Extracting data for each report
extracted_data = [extract_report_data(report) for report in reports]

# Creating a DataFrame from the extracted data
df_reports = pd.DataFrame(extracted_data)



In [110]:
# Displaying the first few rows of the DataFrame
df_reports.head(4)

,ID,REPORTDATE,REFERENCEID,REPORTSOURCE,REPORTDESCRIPTION,PERSONS,DATES,PLACES,ORGANIZATIONS
0,FBI_1,4/28/2003,CIA_7,Canadian Security Intelligence Service,This report concerns a bulletin issued on 2 Ap...,Abdillah Zinedine;Abu Hafs;Mehdi Rafiki;Patric...,4/2/2003;4/1/2003;4/6/2003,/Paris/ /France;175 Rue Durocher/Montreal/Queb...,Surete' de l'Etat;Canadauto Car Rental Agency
1,FBI_2,5/12/2003,,"MI-5,UK","Riyad Yasser, a UK citizen, was arrested on 1 ...",Riyad Yasser;Omar Mahmoud Othman,5/1/2003,M4 Motorway/London/ /UK;721 St. Clare St./Mont...,Baker St. Mosque
2,FBI_3,7/3/2003,CIA_13,Los Angeles Field Office,"In response to the CIA report [29 June, 2003] ...",Abu Somad;Yazid Bafaba;Riduan Sungkar;Omar Eve...,,235 Buckthorn St./Inglewood/California/USA;773...,BMI;CalNational Bank
3,FBI_4,10/21/2003,,Chicago Field Office,A man using an Illinois driver's license in th...,Abu Hafs;Mehdi Rafiki,9/27/2003;10/2/2003;4/27/2003,2211 Cermak Rd./Chicago/Illinois/USA;Hobby Air...,Drexel Car Rental Agency;Central States Airline


In [111]:
Dataset_report_descriptions = df_reports[['REPORTDESCRIPTION']]

In [112]:
Dataset_report_descriptions.head(4)

,REPORTDESCRIPTION
0,This report concerns a bulletin issued on 2 Ap...
1,"Riyad Yasser, a UK citizen, was arrested on 1 ..."
2,"In response to the CIA report [29 June, 2003] ..."
3,A man using an Illinois driver's license in th...


In [113]:
# Reading the Dataset
file_path = 'Terrorism_Related_Words.xlsx'

Terrorism_Related_Words = pd.read_excel(file_path)

In [114]:
Terrorism_Related_Words

,Word,Threat Level
0,bombing,5
1,kidnapping,2
2,assassination,5
3,suicide attack,5
4,mass shooting,5
...,...,...
471,extremist,2
472,radical,2
473,insurgent,2
474,rebel,2


In [115]:
# KeyBERT:

In [ ]:
pip install keybert

In [117]:
from keybert import KeyBERT

kw_model = KeyBERT()

kw_model_multi = KeyBERT('paraphrase-multilingual-MiniLM-L12-v2')

In [118]:
all_keywords = []

for text in Dataset_report_descriptions["REPORTDESCRIPTION"]:
    temp = kw_model.extract_keywords(text, top_n=20)
    all_keywords.append(temp)

In [119]:
Dataset_report_descriptions["REPORTDESCRIPTION"][0]

"This report concerns a bulletin issued on 2 April, 2003 by the Surete' de l'Etat in Paris concerning a Moroccan named Abdillah Zinedine, alias: Abu Hafs [See CIA report for 17 April, 2003].The Surete' reported that Zinedine left Paris on a flight to Montreal, Canada on 1 April, 2003 and was traveling under a French passport in the name of Mehdi Rafiki. A Mehdi Rafiki did arrive in Montreal on 2 April, 2003 and said the purpose of his visit was to attend the funeral of an uncle. He listed his Montreal contact address as 175 Rue Durocher. This address belongs to an Irish pub operated by a man named Patrick O'alley. O'alley says he never heard of any person named Mehdi Rafiki. But a man using a French passport in the name Abu Hafs rented a car at the Canadauto Car Rental Agency in Montreal on 2 April, 2003. This rented car was never returned to this agency. It was discovered abandoned on Walden Ave near Schiller Park in Buffalo, NY on 6 April, 2003"

In [120]:
all_keywords[0]

[('zinedine', 0.4175),
 ('mehdi', 0.3788),
 ('rue', 0.3572),
 ('paris', 0.3568),
 ('durocher', 0.3155),
 ('canadauto', 0.3021),
 ('montreal', 0.2856),
 ('french', 0.2788),
 ('abandoned', 0.2773),
 ('address', 0.2719),
 ('moroccan', 0.265),
 ('passport', 0.2635),
 ('abdillah', 0.263),
 ('rafiki', 0.2524),
 ('etat', 0.2482),
 ('cia', 0.2431),
 ('abu', 0.2392),
 ('named', 0.2265),
 ('person', 0.226),
 ('alias', 0.2239)]

In [121]:
len(all_keywords)

111

In [122]:
len(all_keywords[0])

20

In [123]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering

# Load pre-trained sentence transformer model
s_model = SentenceTransformer('all-MiniLM-L6-v2')

In [124]:
#embedding for Terrorism_Related_Words:

In [125]:
threat_embeddings = s_model.encode(list(Terrorism_Related_Words['Word']))

In [126]:
len(threat_embeddings)

476

In [127]:
#embedding for Dataset key words:

In [128]:
Terrorism_Related_Words

,Word,Threat Level
0,bombing,5
1,kidnapping,2
2,assassination,5
3,suicide attack,5
4,mass shooting,5
...,...,...
471,extremist,2
472,radical,2
473,insurgent,2
474,rebel,2


In [129]:
all_keywords[0]

[('zinedine', 0.4175),
 ('mehdi', 0.3788),
 ('rue', 0.3572),
 ('paris', 0.3568),
 ('durocher', 0.3155),
 ('canadauto', 0.3021),
 ('montreal', 0.2856),
 ('french', 0.2788),
 ('abandoned', 0.2773),
 ('address', 0.2719),
 ('moroccan', 0.265),
 ('passport', 0.2635),
 ('abdillah', 0.263),
 ('rafiki', 0.2524),
 ('etat', 0.2482),
 ('cia', 0.2431),
 ('abu', 0.2392),
 ('named', 0.2265),
 ('person', 0.226),
 ('alias', 0.2239)]

In [140]:
from sentence_transformers import util

In [165]:
# countingh the number of people:
def count_names(names_string):
    if names_string and names_string.strip():
        names_list = names_string.split(';')
        return len(names_list)
    else:
        return 0

In [204]:
# Find the most similar words from threat_levels for each word in data_keys

final_threat_score = []
final_threat_score
dic = {}

for j, keys in enumerate(all_keywords):
    dic_temp = {}
    keys_words = [t[0] for t in keys]
    embedded_keys = s_model.encode(keys_words)
    sum = 0
    temp_level = []
    each_word_score = []
    for i, word in enumerate(keys_words):
        word_score = []
        # Compute cosine similarity between word embedding and threat embeddings
        similarities = util.cos_sim(embedded_keys[i], threat_embeddings)
        # Find the index of the highest similarity score
        most_similar_idx = similarities[0].argmax().item()
        # Get the similarity score
        similarity_score = similarities[0][most_similar_idx].item()
        # Get the most similar word and its threat level
        most_similar_word = Terrorism_Related_Words['Word'][most_similar_idx]
        threat_level =  Terrorism_Related_Words['Threat Level'][most_similar_idx]
        # Print the most similar word, its threat level, and the similarity score

        if similarity_score < 0.6:
            word_score = round(threat_level * similarity_score)
            sum = sum + word_score
        else:
            word_score = round(threat_level * threat_level)
            sum = sum + word_score

        each_word_score.append(word_score)

    final_threat_score.append(sum)

    dic_temp['ID'] = j
    dic_temp['Report'] = df_reports['REPORTDESCRIPTION'][j]
    dic_temp['Threat_Score_Person_0'] = sum
    dic_temp['Threat_Score_Person_3'] = sum + (3 * count_names(df_reports['PERSONS'][j]))
    dic_temp['Threat_Score_Person_6'] = sum + (6 * count_names(df_reports['PERSONS'][j]))
    dic_temp['Threat_Score_Person_10'] = sum + (10 * count_names(df_reports['PERSONS'][j]))
    dic_temp['Persons_Num'] = count_names(df_reports['PERSONS'][j])
    dic_temp['Persons'] = df_reports['PERSONS'][j]
    dic_temp['Places'] = df_reports['PLACES'][j]
    dic_temp['Date'] = df_reports['REPORTDATE'][j]
    dic_temp['Keywords'] = keys_words
    dic_temp['Keywords_Score'] = each_word_score

    dic[j] = dic_temp


In [205]:
answer = final_threat_score.index(max(final_threat_score))
print(answer)

11


In [206]:
Dataset_report_descriptions["REPORTDESCRIPTION"][answer]

"A tip from an anonymous source directed investigators to a residence at 2111 Troost Ave. in Kansas City, Missouri. This source said that explosives were being manufactured at this residence by some terrorists who intended to blow up several buildings in the Kansas City Area. A warrant was obtained to search this residence on evidence provided by an examination of items found in garbage sacks that were picked up by an FBI officer posing as a garbage collector. Traces of chemicals necessary to produce explosives were found on items in this garbage. When this residence was searched on 22 December, 2003, one man, who gave his name as Walter Smith, was the only person at home. While the investigation was proceeding, another man entered and was apprehended. Subsequent identity checks revealed this man to be Jamal al Quso, who had come from Chicago to Kansas City. What was found at this residence were batches of hexamethylene triperoxide diamine [HMTD] and cyclotrimethylene trinitramine [RDX

In [207]:
import heapq

heapq.nlargest(4, final_threat_score)

[102, 94, 78, 72]

In [208]:
answers = [final_threat_score.index(x) for x in heapq.nlargest(5, final_threat_score)]
answers

[11, 52, 71, 33, 73]

In [209]:
_ = [print(Dataset_report_descriptions["REPORTDESCRIPTION"][x], "\n") for x in answers]

A tip from an anonymous source directed investigators to a residence at 2111 Troost Ave. in Kansas City, Missouri. This source said that explosives were being manufactured at this residence by some terrorists who intended to blow up several buildings in the Kansas City Area. A warrant was obtained to search this residence on evidence provided by an examination of items found in garbage sacks that were picked up by an FBI officer posing as a garbage collector. Traces of chemicals necessary to produce explosives were found on items in this garbage. When this residence was searched on 22 December, 2003, one man, who gave his name as Walter Smith, was the only person at home. While the investigation was proceeding, another man entered and was apprehended. Subsequent identity checks revealed this man to be Jamal al Quso, who had come from Chicago to Kansas City. What was found at this residence were batches of hexamethylene triperoxide diamine [HMTD] and cyclotrimethylene trinitramine [RDX]

In [210]:
import json

with open("Dataset.json", "w") as outfile:
    json.dump(dic, outfile)

In [211]:
print("The End")

The End
